<a href="https://colab.research.google.com/github/sai-raman/seq2seq-chatbot/blob/main/ultron_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seq2Seq Chat Bot with Tensorflow & Keras | Encoder**

In [270]:
import pandas as pd
import tensorflow as tf


In [277]:
lines = open('movie_lines.txt',encoding="utf-8",errors="ignore").read().split('\n')
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [278]:
conversations = open('movie_conversations.txt',encoding="utf-8",errors="ignore").read().split('\n')
conversations[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

# **Data Preprocessing**

In [239]:

exchanges = []
for conver in conversations:
    exchanges.append(conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())

In [240]:
dialogs = {}
for line in lines:
  dialogs[line.split(' +++$+++ ')[0]] = line.split(' +++$+++ ')[-1]

In [280]:
dialogs # Dictionary

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [241]:
questions = []
answers = []

for conver in exchanges:
    for i in range(len(conver) - 1):
        questions.append(dialogs[conver[i]])
        answers.append(dialogs[conver[i+1]])


In [242]:
sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 13:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])

In [243]:
import re

def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm",'i am',text)
  text = re.sub(r"he's",'he is',text)
  text = re.sub(r"she's",'she is',text)
  text = re.sub(r"that's",'that is',text)
  text = re.sub(r"what's",'what is',text)
  text = re.sub(r"where's",'where is',text)
  text = re.sub(r"ll",'will',text)
  text = re.sub(r"'ve",'have',text)
  text = re.sub(r"'re",'are',text)
  text = re.sub(r"'d",'would',text)
  text = re.sub(r"wont",'will not',text)
  text = re.sub(r"can't",'cannot',text)
  text = re.sub(r"[^\w\s]",'',text)
  return text



In [244]:
clean_questions = []
clean_answers = []

for line in sorted_ques:
    clean_questions.append(clean_text(line))
        
for line in sorted_ans:
    clean_answers.append(clean_text(line))


In [245]:

for i in range(len(clean_answers)):
    clean_answers[i] = ' '.join(clean_answers[i].split()[:11])

In [281]:
clean_questions[:10]

['cameron',
 'why',
 'there',
 'sure have',
 'hi',
 'i was',
 'wewill no',
 'but',
 'what crap',
 'no']

In [247]:
word2count = {}

for line in clean_questions:
  for word in line.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word]+=1



for line in clean_answers:
  for word in line.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word]+=1

In [276]:
len(word2count)

16443

In [249]:
threshold = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= threshold:
        vocab[word] = word_num
        word_num += 1


In [250]:
for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'

In [251]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [252]:
inv_vocab = {w:v for v, w in vocab.items()}

In [253]:

encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)


In [254]:
decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)


In [255]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 13, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 13, padding='post', truncating='post')

In [256]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 13, padding='post', truncating='post')


In [257]:

from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

# **Building Model using LSTM**

In [258]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input

In [259]:


enc_inp = Input(shape=(13, ))
dec_inp = Input(shape=(13, ))


In [260]:


VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=13,
                  trainable=True                  
                  )



In [262]:


enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]



dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)




model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=40)

Epoch 1/40
938/938 [==============================] - 14s 15ms/step - loss: 2.9573 - acc: 0.5190
Epoch 2/40
938/938 [==============================] - 14s 15ms/step - loss: 2.6470 - acc: 0.5440
Epoch 3/40
938/938 [==============================] - 14s 15ms/step - loss: 2.5651 - acc: 0.5491
Epoch 4/40
938/938 [==============================] - 14s 15ms/step - loss: 2.5131 - acc: 0.5517
Epoch 5/40
938/938 [==============================] - 14s 15ms/step - loss: 2.4550 - acc: 0.5545
Epoch 6/40
938/938 [==============================] - 14s 15ms/step - loss: 2.4040 - acc: 0.5572
Epoch 7/40
938/938 [==============================] - 14s 15ms/step - loss: 2.3546 - acc: 0.5597
Epoch 8/40
938/938 [==============================] - 14s 15ms/step - loss: 2.3054 - acc: 0.5624
Epoch 9/40
938/938 [==============================] - 14s 15ms/step - loss: 2.2555 - acc: 0.5647
Epoch 10/40
938/938 [==============================] - 14s 15ms/step - loss: 2.2059 - acc: 0.5671
Epoch 11/40
938/938 [========

# **Inference Model to make predictions from encoder to decoder model**

In [263]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


In [264]:

enc_model = Model([enc_inp], enc_states)

In [265]:

# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)


In [266]:
from keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")



##########################################
#       start chatting ver. 1.0          #
##########################################


In [269]:
prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 13, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================") 

you : hey
chatbot attention :  hey 
you : how are you
chatbot attention :  fine 
you : who are you
chatbot attention :  i am the <OUT> 
you : what are you
chatbot attention :  i am not sure you said i wasnt <OUT> or fear 
you : did you vote
chatbot attention :  yes 
you : when
chatbot attention :  <OUT> 


KeyboardInterrupt: ignored